In [1]:
!pip install ultralytics

In [7]:
import ultralytics
print("Ultralytics version:", ultralytics.__version__)

Ultralytics version: 8.3.202


In [ ]:
import torch

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="2gDGLLPIGMEJfyRDX8OS")
project = rf.workspace("burtiwayne").project("iicence_plate_detector")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Iicence_plate_detector-1 in yolov8:: 100%|██████████| 1486/1486 [00:00<00:00, 7094.51it/s]


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving best (4).pt to best (4).pt


In [3]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data="/content/Iicence_plate_detector-1/data.yaml",
    epochs=30,
    imgsz=800,
    name="license_plate_detector"
)
metrics = model.val()
print(metrics)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Iicence_plate_detector-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license_plate_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

In [6]:
from google.colab import files
import os

model_path = '/content/runs/detect/license_plate_detector/weights/best.pt'

if os.path.exists(model_path):
  files.download(model_path)
else:
  print(f"Model file not found at {model_path}. Please check the path.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
from google.colab import files
uploaded = files.upload()  # Upload 'best.pt'

Saving best.pt to best (1).pt


In [7]:
!apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
from ultralytics import YOLO

# Load the trained model
model = YOLO("/content/best (1).pt")

# Validate the model on a dataset
metrics = model.val(data="/content/Iicence_plate_detector-1/data.yaml")

# Print mAP50 and mAP50-95
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

# Print per-class precision and recall
# metrics.box.p contains a list of precision values for each class
# metrics.box.r contains a list of recall values for each class
print(f"Precision per class: {metrics.box.p}")
print(f"Recall per class: {metrics.box.r}")

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1699.0±370.3 MB/s, size: 50.4 KB)
val: Scanning /content/Iicence_plate_detector-1/valid/labels.cache... 110 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 110/110 226.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.6it/s 4.3s
                   all        110        137      0.896      0.891      0.898      0.447
Speed: 8.3ms preprocess, 9.3ms inference, 0.0ms loss, 5.5ms postprocess per image
Results saved to /content/runs/detect/val
mAP50: 0.8977
mAP50-95: 0.4467
Precision per class: [    0.89581]
Recall per class: [    0.89051]


In [35]:
!pip install easyocr

import cv2
import easyocr
from ultralytics import YOLO

# Load YOLOv8 trained model
yolo_model = YOLO("/content/best (1).pt")

# Load EasyOCR Reader (English for license plates)
reader = easyocr.Reader(['en'])

# Input image
img_path = "/content/20250223_141324_jpg.rf.997031e729146f638b1693dd8ff9a595.jpg"
results = yolo_model(img_path)

for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        crop = cv2.imread(img_path)[y1:y2, x1:x2]  # Crop license plate
        ocr_result = reader.readtext(crop)

        # Extract detected text
        text = " ".join([res[1] for res in ocr_result])
        print("Detected Plate:", text)



image 1/1 /content/20250223_141324_jpg.rf.997031e729146f638b1693dd8ff9a595.jpg: 800x800 1 licence_plate, 9.7ms
Speed: 3.5ms preprocess, 9.7ms inference, 1.9ms postprocess per image at shape (1, 3, 800, 800)
Detected Plate: 87 4 4 5616


In [ ]:
!pip install easyocr ultralytics opencv-python

import cv2
import easyocr
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Load YOLOv8 trained model
yolo_model = YOLO("/content/best (1).pt")

# Load EasyOCR Reader (English for license plates)
reader = easyocr.Reader(['en'])

# Input image
img_path = "/content/1000145617.jpg"
results = yolo_model(img_path)

# Load the original image for annotation
frame = cv2.imread(img_path)

# Zoom factor (increase bounding box by 10% on each side)
zoom_factor = 0.1

for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)

        # Calculate zoomed-in coordinates
        width = x2 - x1
        height = y2 - y1
        x1_zoom = max(0, int(x1 - zoom_factor * width))
        y1_zoom = max(0, int(y1 - zoom_factor * height))
        x2_zoom = min(frame.shape[1], int(x2 + zoom_factor * width))
        y2_zoom = min(frame.shape[0], int(y2 + zoom_factor * height))

        # Draw bounding box (original coordinates)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Crop the license plate region with zoom
        crop = frame[y1_zoom:y2_zoom, x1_zoom:x2_zoom]

        # OCR recognition
        ocr_result = reader.readtext(crop)
        text = " ".join([res[1].upper() for res in ocr_result])  # Convert to uppercase

        # Put OCR text above the bounding box with bigger font
        cv2.putText(frame, text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)

        print("Detected Plate:", text)

# Save the result image
cv2.imwrite("output_with_ocr_zoom.jpg", frame)

# Show image in Colab
cv2_imshow(frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
!pip install easyocr ultralytics opencv-python

import cv2
import easyocr
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Load YOLOv8 trained model
yolo_model = YOLO("/content/best (1).pt")

# Load EasyOCR Reader (English for license plates)
reader = easyocr.Reader(['en'])

# Input image
img_path = "/content/1000145617.jpg"
results = yolo_model(img_path)

# Load the original image for annotation
frame = cv2.imread(img_path)

# Zoom factor (increase bounding box by 10% on each side)
zoom_factor = 0.1

for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)

        # Calculate zoomed-in coordinates
        width = x2 - x1
        height = y2 - y1
        x1_zoom = max(0, int(x1 - zoom_factor * width))
        y1_zoom = max(0, int(y1 - zoom_factor * height))
        x2_zoom = min(frame.shape[1], int(x2 + zoom_factor * width))
        y2_zoom = min(frame.shape[0], int(y2 + zoom_factor * height))

        # Draw bounding box (original coordinates)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Crop the license plate region with zoom
        crop = frame[y1_zoom:y2_zoom, x1_zoom:x2_zoom]

        # OCR recognition
        ocr_result = reader.readtext(crop)

        # Sort OCR results by row (y-coordinate) and right-to-left in each row
        if ocr_result:
            # Sort by y (top to bottom)
            ocr_result.sort(key=lambda x: x[0][0][1])  # use top-left y-coordinate

            # Group by rows (tolerance for same row)
            rows = []
            current_row = []
            row_tolerance = 10  # pixels
            prev_y = ocr_result[0][0][0][1]

            for res in ocr_result:
                y_top = res[0][0][1]
                if abs(y_top - prev_y) <= row_tolerance:
                    current_row.append(res)
                else:
                    rows.append(current_row)
                    current_row = [res]
                    prev_y = y_top
            rows.append(current_row)

            # For each row, sort right-to-left (descending x-coordinate)
            final_text = []
            for row in rows:
                row.sort(key=lambda x: x[0][0][0], reverse=True)  # x-coordinate of top-left
                row_text = " ".join([r[1].upper() for r in row])
                final_text.append(row_text)

            text = "\n".join(final_text)  # Join rows with newline
        else:
            text = ""

        # Put OCR text above the bounding box with bigger font
        for i, line in enumerate(final_text):
            cv2.putText(frame, line, (x1, y1 - 10 - i*40),  # 40 px per line
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)

        print("Detected Plate:\n", text)

# Save the result image
cv2.imwrite("output_with_ocr_rtl.jpg", frame)

# Show image in Colab
cv2_imshow(frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
